# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')
FATABLE = M1M3FATable.FATABLE

## Plot the actuator locations

In [ ]:
def actuatorLayout(ax, FATABLE):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 Actuator positions and type\nHardpoints are approximate", fontsize=18)
    types = [['SAA','NA', 'o', 'Z', 'b'], ['DAA','+Y', '^', '+Y','g'], ['DAA','-Y', 'v', '-Y', 'cyan'], \
             ['DAA','+X', '>', '+X', 'r'], ['DAA','-X', '<', '-X', 'r']]
    for [type, orient, marker, label, color] in types:
        xs = []
        ys = []
        for i in range(len(FATABLE)):
            x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
            y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
            if FATABLE[i][M1M3FATable.FATABLE_TYPE] == type and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == orient:
                xs.append(x)
                ys.append(y)
            else:
                continue
        ax.scatter(xs, ys, marker=marker, color=color, s=200, label=label)

    # Now plot approximate hardpoint location
    Rhp = 3.1 # Radius in meters
    for i in range(6):
        theta = 2.0 * np.pi / 6.0 * float(i)
        if i == 0:
            ax.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', \
                       s=200, label='HP')
        else:
            ax.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', \
                       s=200, label='_nolegend_')
    ax.legend(loc='lower left', fontsize=9)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
actuatorLayout(ax, FATABLE)

# Now plot some actual bump test results

In [ ]:
# Times of bump test
start = Time("2023-04-17T10:00:00", scale='utc')
end = Time("2023-04-17T11:15:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(bumps)

## Plot the Bump Test states

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

id=327
thisBump = bumps[bumps['actuatorId']==id]
index = M1M3FATable.actuatorIDToIndex(id)
primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
fig, ax = plt.subplots(1,1)
thisBump[primaryBump].plot(ax=ax)
ax. set_title(f" Bump Test States id = {id}")
ax.set_ylim(0,7)
ax2 = ax.twinx()
ax2.set_yticks([0,1,2,3,4,5,6,7])
ax2.set_yticklabels(['', 'NOTTESTED','TESTINGPOSITIVE','TESTINGPOSITIVEWAIT', \
                   'TESTINGNEGATIVE', 'TESTINGNEGATIVEWAIT','PASSED','FAILED'])
plt.savefig("/home/craiglagegit/u/MTM1M3/data/technote/Bump_Test_States.png")

## Given an actuator ID, this plots the bump applied forces

In [ ]:
async def plotBumpTestAppliedForces(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForces{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForces{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForces{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(1.0, 350.0, "lsst.sal.MTM1M3.appliedForces."+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(1.0, 350.0, "lsst.sal.MTM1M3.appliedForces."+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestAppliedForces(fig, bumps, 327)
plt.savefig("/home/craiglagegit/u/MTM1M3/data/technote/Bump_Test_Target.png")

## Given an actuator ID, this plots the bump test result

In [ ]:
async def plotBumpTestResults(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(0.5, 350.0, "lsst.sal.MTM1M3.forceActuatorData."+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(0.5, 350.0, "lsst.sal.MTM1M3.forceActuatorData."+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestResults(fig, bumps, 327)
plt.savefig("/home/craiglagegit/u/MTM1M3/data/technote/Bump_Test_Results.png")

## Given an actuator ID, this plots the cylinder applied forces

In [ ]:
async def plotBumpTestAppliedCylinderForces(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"primaryCylinderForces{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"secondaryCylinderForces{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 28.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.appliedCylinderForces", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(1.0, 340000.0, "lsst.sal.MTM1M3.appliedCylinderForces.\n"+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400000,400000)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (mNt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
        #plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 28.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.appliedCylinderForces", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(1.0, 340000.0, "lsst.sal.MTM1M3.appliedCylinderForces.\n"+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400000,400000)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (mNt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestAppliedCylinderForces(fig, bumps, 327)
plt.savefig("/home/craiglagegit/u/MTM1M3/data/technote/Bump_Test_Cylinder_Target.png")

## Given an actuator ID, this plots the cylinder measured forces

In [ ]:
async def plotBumpTestCylinderForces(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"primaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"secondaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 28.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(1.0, 340.0, "lsst.sal.MTM1M3.forceActuatorData.\n"+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (Nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
        #plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 28.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(1.0, 340.0, "lsst.sal.MTM1M3.forceActuatorData.\n"+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (Nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestCylinderForces(fig, bumps, 327)
plt.savefig("/home/craiglagegit/u/MTM1M3/data/technote/Bump_Test_Cylinder_Results.png")

## Given an actuator ID, this plots the bump test following errors

In [ ]:
async def plot_bump_test_following_errors(fig, bumps, id):
    """ Plot a visualization of the bump test following errors
        Parameters
        ----------
        fig : a matplotlib figure object

        bumps: pandas dataframe
            This is a dataframe containg the bump test status
        
        id: 'int'
            The actuator id desired

        Returns
        -------
        No return, only the fig object which was input
    """
    
    this_bump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primary_bump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primary_follow = f"primaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primary_force = f"primaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        secondary_bump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondary_force = f"secondaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondary_follow = f"secondaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondary_name = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondary_name = None
        secondary_force = None
        secondary_follow = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}", fontsize=18)
    plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - 1.0
    plot_end = plot_start + 14.0 
    start = Time(plot_start, format='unix_tai', scale='tai')
    end = Time(plot_end, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primary_force, primary_follow, 'timestamp'], start.utc, end.utc)
    timestamp = forces.index[0].isoformat().split('.')[0]
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}\n {timestamp}", fontsize=18)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primary_forces = forces[primary_force].values
    primary_errors = forces[primary_follow].values
    plot_start -= t0
    plot_end -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, primary_forces, label='Data')
    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.title("Following Errors - Log scale above 10 Nt")
    plt.plot(times, primary_errors)
    plt.ylim(-100,100)
    plt.yscale('symlog', linthresh=10)
    plt.plot([plot_start, plot_end], [5.0,5.0], ls='--', color='red')
    plt.plot([plot_start, plot_end], [-5.0,-5.0], ls='--', color='red')
    plt.plot([plot_start, plot_end], [2.5,2.5], ls='--', color='green')
    plt.plot([plot_start, plot_end], [-2.5,-2.5], ls='--', color='green')
    plt.text(9, 65, f"Max = {np.max(primary_errors):.1f} nt")
    plt.text(5, -80, f"Min = {np.min(primary_errors):.1f} nt")
    plt.xlim(plot_start, plot_end)
    plt.yticks([-100,-10,-7.5,-5.0,-2.5,0,2.5,5.0,7.5,10.0,100])
    plt.xlabel("Time (seconds)")
    plt.ylabel("Following Errors (nt)")
    
    if secondary_name is not None:
        plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[0] - 1.0
        plot_end = plot_start + 14.0
        start = Time(plot_start, format='unix_tai', scale='tai')
        end = Time(plot_end, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [secondary_force, secondary_follow, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondary_forces = forces[secondary_force].values
        secondary_errors = forces[secondary_follow].values
        plot_start -= t0
        plot_end -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondary_name}")
        plt.plot(times, secondary_forces, label='Data')
        plt.xlim(plot_start, plot_end)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
        plt.legend()
        plt.subplot(2,2,4)
        plt.title("Following Errors - Log scale above 10 Nt")
        plt.plot(times, secondary_errors)
        plt.yscale('symlog', linthresh=10)
        plt.plot([plot_start, plot_end], [5.0,5.0], ls='--', color='red')
        plt.plot([plot_start, plot_end], [-5.0,-5.0], ls='--', color='red')
        plt.plot([plot_start, plot_end], [2.5,2.5], ls='--', color='green')
        plt.plot([plot_start, plot_end], [-2.5,-2.5], ls='--', color='green')
        plt.text(9, 65, f"Max = {np.max(secondary_errors):.1f} nt")
        plt.text(5, -80, f"Min = {np.min(secondary_errors):.1f} nt")
        plt.xlim(plot_start, plot_end)
        plt.ylim(-100, 100)
        plt.yticks([-100,-10,-7.5,-5.0,-2.5,0,2.5,5.0,7.5,10.0,100])
        plt.xlabel("Time (seconds)")
        plt.ylabel("Following Errors (nt)")
    else:
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
start = "2023-07-04T03:00:00"
end = "2023-07-04T22:00:00"
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                        Time(start, scale='utc'), Time(end, scale='utc'))

fig = plt.figure(figsize=(10,10))
await plot_bump_test_following_errors(fig, bumps, 327)
plt.savefig("/home/craiglagegit/u/MTM1M3/data/technote/Bump_Test_Following_Errors.png")

## The cells below give examples of the actuator indexing

In [ ]:
index = M1M3FATable.actuatorIDToIndex(327)
print(index)

In [ ]:
FATABLE[index][M1M3FATable.FATABLE_INDEX]

In [ ]:
FATABLE[index][M1M3FATable.FATABLE_ID]

In [ ]:
FATABLE[index][M1M3FATable.FATABLE_SINDEX]

In [ ]:
FATABLE[index][M1M3FATable.FATABLE_ZINDEX]

In [ ]:
FATABLE[index][M1M3FATable.FATABLE_YINDEX]

In [ ]:
index = M1M3FATable.actuatorIDToIndex(135)
print(index)

In [ ]:
FATABLE[index][M1M3FATable.FATABLE_XINDEX]